**비디오에서 얼굴 검출하기**

In [ ]:
import cv2 as cv
import mediapipe as mp

#mediapipe에서 제공하는 얼굴 감지(눈,코,입 위치) 모듈
mp_face_detection=mp.solutions.face_detection
#감지된 얼굴의 위치나 특징점을 화면에 그려주는 기능
mp_drawing=mp.solutions.drawing_utils

#얼굴 감지 객체
#model_selection(0:근거리, 1:원거리)
#min_detection_confidence(얼굴로 인정할 최소 신뢰도)
face_detection=mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

#openCV에서 비디오 키기
cap=cv.VideoCapture(0,cv.CAP_DSHOW)

while True:
    #ret: 프레임 읽기 성공 여부
    ret,frame=cap.read()
    if not ret:
        print('프레임 획득 실패')
        break
    #mediapipe는 RGB형식만 지원
    #변환된 영상을 process에 전달
    res=face_detection.process(cv.cvtColor(frame,cv.COLOR_BGR2RGB))
    
    #얼굴이 감지되었을 때
    if res.detections:
        #감지된 얼굴을 하나씩 순회
        for detection in res.detections:
            #감지된 얼굴의 위치와 특징점(눈,코,입 등)을 화면 위에 시각적으로 표시
            mp_drawing.draw_detection(frame,detection)

    #flip(frame,1): 좌우 반전/ 0:상하 반전    
    cv.imshow('MediaPipe Face Detection from video',cv.flip(frame,1))
    if cv.waitKey(5)==ord('q'):
        break

#웹캠 장치 해제    
cap.release()
cv.destroyAllWindows()

**얼굴을 장식하는 증강 현실 구현하기**

| 패키지                       | 버전            |
| ------------------------- | ------------- |
| **python**                | 3.9 (가상환경 버전) |
| **tensorflow**            | 2.10.0        |
| **keras**                 | 2.10.0        |
| **protobuf**              | 3.20.3        |
| **mediapipe**             | 0.10.9        |
| **pixellib**              | 0.7.1         |
| **opencv-python**         | 4.5.5.64      |
| **opencv-contrib-python** | 4.11.0.86     |
| **numpy**                 | 1.26.4        |
| **pillow**                | 11.3.0        |
| **imantics**              | 0.1.12        |
| **matplotlib**            | 3.9.4         |


In [ ]:
import cv2 as cv
import mediapipe as mp

#IMREAD_UNCHANGED: 원본 그대로 읽기(투명도 포함)
img=cv.imread('heart.png',cv.IMREAD_UNCHANGED)
#이미지 크기를 직접 지정하지 않고 10%로 축소
img=cv.resize(img,dsize=(0,0),fx=0.1,fy=0.1)
#shape[1]:이미지 너비, shape[0]:높이
w,h=img.shape[1],img.shape[0]

#mediapipe에서 제공하는 얼굴 감지(눈,코,입 위치) 모듈
mp_face_detection=mp.solutions.face_detection
#감지된 얼굴의 위치나 특징점을 화면에 그려주는 기능
mp_drawing=mp.solutions.drawing_utils


#얼굴 감지 객체
#model_selection(0:근거리, 1:원거리)
#min_detection_confidence(얼굴로 인정할 최소 신뢰도)
face_detection=mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

cap=cv.VideoCapture(0,cv.CAP_DSHOW)

while True:
    ret,frame=cap.read()
    #ret: 프레임 읽기 성공 여부
    if not ret:
        print('프레임 획득 실패')
        break
    
    #mediapipe는 RGB형식만 지원
    #변환된 영상을 process에 전달
    res=face_detection.process(cv.cvtColor(frame,cv.COLOR_BGR2RGB))
    
    #얼굴이 감지되었을 때
    if res.detections:
        #감지된 얼굴을 하나씩 순회
        for det in res.detections:
            #양쪽 눈 좌표 가져오기
            for eye_point in [mp_face_detection.FaceKeyPoint.RIGHT_EYE, mp_face_detection.FaceKeyPoint.LEFT_EYE]:
                p = mp_face_detection.get_key_point(det, eye_point)
                #감지된 눈 좌표(p.x)를 프레임 픽셀 좌표(가로 크기)로 변환(*frame.shape[1])
                #불러온 이미지 중심이 눈 좌표에 오도록(-w//2)
                x1,x2=int(p.x*frame.shape[1]-w//2),int(p.x*frame.shape[1]+w//2)
                y1,y2=int(p.y*frame.shape[0]-h//2),int(p.y*frame.shape[0]+h//2)
                #불러온 이미지가 프레임을 벗어나지 않을 때
                if x1>0 and y1>0 and x2<frame.shape[1] and y2<frame.shape[0]:
                    #이미지가 png이므로 RGBA(4채널)
                    #/255: 알파 채널(투명도)을 0~1로 정규화 (alpha=1.0:불투명, 0.0: 완전 투명)
                    alpha=img[:,:,3:]/255
                    #눈 영역(frame[y1:y2,x1:x2]) * (1-alpha): alpha값이 0~1일 때 배경 픽셀과 불러온 이미지 픽셀이 혼합, alpha가 1이면 불러온 이미지가 완전히 덮음
                    #img[:,:,:3]: RGB채널만 추출 * alpha = 불러온 이미지의 불투명한 부분은 진하게, 투명한 부분은 연하게 적용
                    frame[y1:y2,x1:x2]=frame[y1:y2,x1:x2]*(1-alpha)+img[:,:,:3]*alpha
                
    cv.imshow('MediaPipe Face AR',cv.flip(frame,1))
    if cv.waitKey(5)==ord('q'):
        break

cap.release()
cv.destroyAllWindows()    
    